In [1]:
#Cria sessão Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

In [2]:
#Instância Spark
spark = SparkSession.builder.appName('Curso Pyspark') \
         .config('spark.sql.repl.eagerEval.enabled', True) \
         .getOrCreate()      

In [3]:
# lendo um arquivo parquet
df = spark.read.parquet('./DATASETS/COMPRAS.parquet')

In [4]:
df.show(20, truncate=0)

+------+---------------------+-------------------+---------------------------+----------+-------------------------+----------+--------+---------------+---------------------------------------+-----------+---------+----------+
|id    |cartao_data_expiracao|cartao_numero      |cartao_bandeira            |cartao_cvc|codigo_transacao_bancaria|data      |hora    |ipv4           |ipv6                                   |cep_entrega|cd_livro |cd_cliente|
+------+---------------------+-------------------+---------------------------+----------+-------------------------+----------+--------+---------------+---------------------------------------+-----------+---------+----------+
|012389|11/25                |5500804500517692   |Discover                   |959       |GB98MPIH62210859391317   |2021-07-24|03:21:28|62.145.31.164  |b23d:58a2:9eff:3667:47fe:d0bf:9241:433f|36629-219  |030334762|3339828   |
|012476|01/31                |4609489235873      |VISA 16 digit              |6979      |GB79GAVL233

In [8]:
(
    df
    .withColumn('mes', F.date_format('data', 'MMMM'))
    .groupBy('cartao_bandeira', 'mes')
    .agg(F.count('*'))
    
    .show(20, truncate=0)
)

+---------------------------+---------+--------+
|cartao_bandeira            |mes      |count(1)|
+---------------------------+---------+--------+
|Discover                   |June     |297     |
|American Express           |February |353     |
|VISA 19 digit              |July     |352     |
|American Express           |May      |346     |
|VISA 19 digit              |October  |324     |
|American Express           |December |332     |
|VISA 16 digit              |November |616     |
|Mastercard                 |August   |289     |
|JCB 16 digit               |July     |641     |
|JCB 15 digit               |March    |401     |
|Diners Club / Carte Blanche|October  |328     |
|VISA 13 digit              |August   |335     |
|Diners Club / Carte Blanche|April    |300     |
|Discover                   |September|310     |
|JCB 15 digit               |May      |337     |
|VISA 13 digit              |November |302     |
|JCB 15 digit               |February |374     |
|VISA 16 digit      

In [10]:
(
    df
    .withColumn('mes', F.date_format('data', 'MMMM'))
    .groupBy('cartao_bandeira')
    # transformar linhas em colunas
    .pivot('mes', ['January', 'February'])
    .agg(F.count('*'))
    
    .show(20, truncate=0)
)

+---------------------------+-------+--------+
|cartao_bandeira            |January|February|
+---------------------------+-------+--------+
|VISA 16 digit              |878    |772     |
|VISA 13 digit              |420    |350     |
|Discover                   |454    |406     |
|Diners Club / Carte Blanche|397    |395     |
|American Express           |385    |353     |
|Maestro                    |424    |374     |
|Mastercard                 |415    |359     |
|JCB 16 digit               |835    |756     |
|VISA 19 digit              |437    |388     |
|JCB 15 digit               |386    |374     |
+---------------------------+-------+--------+



In [22]:
df2 = (
    df
    .withColumn('mes', F.date_format('data', 'MMMM'))
    .groupBy('cartao_bandeira')
    # transformar linhas em colunas agrupando pelo mes e mostrando a quantidade para cada bandeira
    .pivot('mes', ['January', 'February']) # caso não coloque o segundo parametro automaticamente ele percorre todos os registros
    .agg(F.count('*'))
)

In [23]:

df2

cartao_bandeira,January,February
VISA 16 digit,878,772
VISA 13 digit,420,350
Discover,454,406
Diners Club / Car...,397,395
American Express,385,353
Maestro,424,374
Mastercard,415,359
JCB 16 digit,835,756
VISA 19 digit,437,388
JCB 15 digit,386,374


In [25]:
# aqui estamos desfanzendo e voltando as colunas para linhas
df2.select('cartao_bandeira', F.expr('stack(2, "Jan", January, "Fev", February) as (mes, valor)'))

cartao_bandeira,mes,valor
VISA 16 digit,Jan,878
VISA 16 digit,Fev,772
VISA 13 digit,Jan,420
VISA 13 digit,Fev,350
Discover,Jan,454
Discover,Fev,406
Diners Club / Car...,Jan,397
Diners Club / Car...,Fev,395
American Express,Jan,385
American Express,Fev,353
